## Абстрактные классы

**Абстрактный класс** - класс в котором есть хотя бы один абстрактный метод.   
**Абстрактный метод** - объявленный, но не реализованный метод. 

Абстрактный класс описывает интерфейс взаимодействия с дочерними классами. 

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Animal(ABC): # наследование от ABC из модуля ABC
   
    def __init__(self, name, legs, scariness):
        self.name = name 
        self.legs = legs
        self.scariness = scariness
        
    # общий метод, который будут использовать все наследники этого класса
    def introduce(self): 
        print("Hello! My name is %s!" % self.name)
    
    # абстрактный метод, который будет необходимо переопределять для каждого подкласса
    @abstractmethod # чтобы объявить метод абстрактным используется декоратор @abstactmethod
    def sound(self):
        pass

Абстрактный класс нельзя инстанциировать, обязательно нужно унаследовать и переопределить **все** абстрактные методы в дочернем классе. 

In [ ]:
animal = Animal('Animal', 4, 10)

In [ ]:
class Cat(Animal):
    pass

In [ ]:
cat = Cat('Cat', 4, 2) # унаследовали, но не переопределили абстрактный метод

In [ ]:
class Cat(Animal):
    def sound(self): # переопределяем абстрактный метод
        print('Meow!')

In [ ]:
class Dog(Animal):
    def sound(self): # переопределяем абстрактный метод
        print('Woof!')

In [ ]:
cat = Cat('Cat', 4, 2)
dog = Dog('Dog', 4, 6)

In [ ]:
cat.sound()
dog.sound()

In [ ]:
cat.introduce()
dog.introduce()

## Метаклассы

Вспоминаем, что в питоне все является объектом. В том числе классы. 

In [ ]:
class MyClass: # этот код создает в памяти объект на который ссылается переменная MyClass
    def __init__(self, a):
        print('Created instance of MyClass!')
        self.a = a

Объект `MyClass` может сам порождать объекты --> является классом.

In [ ]:
my_obj = MyClass(1)

При этом с ним можно делать все то же самое, что и с любым объектом:

In [ ]:
# присвоить переменной
class_to_make = MyClass
my_obj = class_to_make(1)

In [ ]:
my_obj.a

In [ ]:
# передать в функцию 
def create_instance(class_object, a=1):
    print(a)
    return class_object(a)

In [ ]:
my_obj = create_instance(MyClass, 1)

In [ ]:
my_obj.a

In [ ]:
# добавить или изменить атрибут (это будет атрибут класса)
MyClass.new_class_attr = 10
my_obj1 = MyClass(1)

In [ ]:
my_obj1.new_class_attr

In [ ]:
MyClass.new_class_attr = 20
my_obj1 = MyClass(2)

In [ ]:
my_obj2.new_class_attr

Классы можно создавать динамически с помощью `type`, передав следующие аргументы: имя нового класса, кортеж с объектами родительских классов, словарь с названиями и значениями атрибутов классов.  

In [ ]:
new_cat_class = type('Kitten', (Cat, ), {'is_smol':True})

In [ ]:
kitten = new_cat_class(name='Kitten', legs=4, scariness=-10)

In [ ]:
kitten.name

In [ ]:
kitten.sound()

In [ ]:
kitten.is_smol

Добавление методов в класс:

In [ ]:
# создаем функцию с нужныи именем и аргументами
def introduce(self):
    return "Hello I am %s, %s year student!" % (self.name, str(self.year))

In [ ]:
# можно создавать классы в цикле
new_classes = []
for i in range(1,5):
    # передаем функцию в словаре атрибутов
    new_classes.append(type('Student_%s_year'%str(i), (object, ), {'year': i, 'introduce': introduce})) 

In [ ]:
names = ['Vasya', 'Masha', 'Petya', 'Dasha']
for i, class_ in enumerate(new_classes):
    obj = class_()
    obj.name = names[i] # атрибуты экземпляра задаются отдельно
    print(obj)
    print(obj.year)
    print(obj.introduce()+'\n')

То же самое можно сделать с помощью метакласса.    
**Метакласс** - класс, экземпляры которого сами являются классами (могут порождать свои экземпляры).  

Попробуем создать классы студентов Х-го курса (такие же как выше), теперь с помощью метакласса. 

In [ ]:
# переопределим конструктор, чтобы атрибут name появлялся сразу после создания объекта
def __init__(self, name):
    self.name = name

In [ ]:
# допустим мы хотим отдельно передавать методы в виде списка
# и так, чтобы они автоматически добавлялись с нужным именем
student_methods = [introduce, __init__]

+ `__new__()` - отвечает за создание нового объекта класса, возвращает новый объект
+ `__init__()` - отвечает за инициализацию нового объекта класса (объявить какие у него могут быть атрибуты)

In [ ]:
class StudentMetaClass(type):
    def __new__(cls, name, bases, attrs):
        for method in attrs['methods']:
            attrs[method.__name__] = method # добавляем пары ключ - название метода, значение -  метод
        attrs.pop('methods') # удаляем methods из словаря атрибутов 
        return super().__new__(cls, name, bases, attrs)

`type` на самом деле тоже является метаклассом, который Python внутренне использует для создания всех классов

In [ ]:
new_classes = []
for i in range(1,5):
    new_classes.append(StudentMetaClass('Student%sYear'%str(i), (object, ), {'year': i, 'methods': student_methods})) 

In [ ]:
names = ['Vasya', 'Masha', 'Petya', 'Dasha']
for i, class_ in enumerate(new_classes):
    obj = class_(name=names[i])
    print(obj)
    print(obj.year)
    print(obj.introduce()+'\n')

### Аргумент metaclass

При написании класса можно добавить аргумент  metaclass, тогда питон при создании класса будет использовать указанный метакласс.   
При указании metaclass питон 
+ перехватывает создание класса
+ изменяет класс
+ возвращает модифицированный объект класса

In [ ]:
class Student1Year(metaclass=StudentMetaClass):
    # задаем атрибуты, такие же как в словаре переданном StudentMetaClass последним аргументом
    year = 1 
    methods = student_methods

In [ ]:
student = Student1Year('Boris')

In [ ]:
student.introduce()

In [ ]:
student.yearclass StudentMetaClass(type):
    def __new__(cls, name, bases, attrs):
        for method in attrs['methods']:
            attrs[method.__name__] = method # добавляем пары ключ - название метода, значение -  метод
        attrs.pop('methods') # удаляем methods из словаря атрибутов 
        return super().__new__(cls, name, bases, attrs)

**Задание**: 
   + написать метакласс, который переводит названия всех атрибутов и методов (кроме служебных) в верхний регистр
   + служебный = начинается и заканчивается на два нижих подчеркивания

In [ ]:
# пример работы
class MyClass(metaclass=UpperCaseMetaclass):
    attr1 = 1

In [ ]:
my_object = MyClass()
my_object.ATTR1
# 1

## Дополнительные материалы

+ [Подробная статья про метаклассы в питоне](https://habr.com/ru/post/145835/)
+ [Метаклассы в продакшене](https://habr.com/ru/company/binarydistrict/blog/422409/)
+ [\_\_init\_\_  vs \_\_new\_\_](https://stackoverflow.com/questions/674304/why-is-init-always-called-after-new)